<a href="https://colab.research.google.com/github/RuiStarlit/MyStudyOfTF-PT/blob/main/%5CLearningPyTorch%5CMyFedavg%5Cstartfedavg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新段落

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import os
path="/content/drive/My Drive/Colab Notebooks/FedAvg/MyFedavg"
os.chdir(path)
os.listdir(path)

['models.py',
 'ResNet_test1log.txt',
 'ResNet_test.py',
 'test_fedavg.py',
 'test1.py',
 'sampling.py',
 'utils.py',
 'fedavg.py',
 '__pycache__',
 'fed_cifar10iid_ResNet18_10_C[0.1]_E[10]_B[50]_loss.png',
 'fed_cifar10iid_ResNet18_10_C[0.1]_E[10]_B[50]_NSHARDS[100]_avg_acc.png',
 'fed_cifar10iid_ResNet18_10_C[0.1]_E[10]_B[50]_NSHARDS[100]_test_acc.png',
 'fed_cifar10iid_ResNet18_50_C[0.1]_E[10]_B[50]_loss.png',
 'fed_cifar10iid_ResNet18_50_C[0.1]_E[10]_B[50]_NSHARDS[100]_avg_acc.png',
 'fed_cifar10iid_ResNet18_50_C[0.1]_E[10]_B[50]_NSHARDS[100]_test_acc.png',
 'fed_cifar10_ResNet18_50_C[0.1]_E[10]_B[50]_loss.png',
 'fed_cifar10_ResNet18_50_C[0.1]_E[10]_B[50]_NSHARDS[100]_avg_acc.png',
 'fed_cifar10_ResNet18_50_C[0.1]_E[10]_B[50]_NSHARDS[100]_test_acc.png',
 'fed_cifar10_ResNet18_50_C[0.1]_E[5]_B[50]_loss.png',
 'fed_cifar10_ResNet18_50_C[0.1]_E[5]_B[50]_NSHARDS[100]_avg_acc.png',
 'fed_cifar10_ResNet18_50_C[0.1]_E[5]_B[50]_NSHARDS[100]_test_acc.png',
 'startfedavg.ipynb']

In [ ]:
!ls

 fedavg.py
'fed_cifar100_ResNet18_10_C[0.1]_E[20]_B[10]_loss.png'
'fed_cifar100_ResNet18_10_C[0.1]_E[20]_B[10]_NSHARDS[200]_avg_acc.png'
'fed_cifar100_ResNet18_10_C[0.1]_E[20]_B[10]_NSHARDS[200]_test_acc.png'
'fed_cifar100_ResNet18_20_C[0.1]_E[20]_B[100]_loss.png'
'fed_cifar100_ResNet18_20_C[0.1]_E[20]_B[100]_NSHARDS[200]_avg_acc.png'
'fed_cifar100_ResNet18_20_C[0.1]_E[20]_B[100]_NSHARDS[200]_test_acc.png'
 models.py
 __pycache__
 ResNet_test1log.txt
 ResNet_test.py
 sampling.py
 test1.py
 test_fedavg.py
 Untitled0.ipynb
 utils.py


In [4]:
# -*- coding:utf-8 -*-
"""
Author: RuiStarlit
File: fedavg
Project: Fedavg
Create Time: 2021-07-07

"""
import os
import copy
import time
import numpy as np
from tqdm import tqdm

import torch

from utils import average_weights, LocalUpdate, get_dataset, test_inference
from models import ResNet18, ResBlock


class Arg:
    def __init__(self):
        self.gpu = 0
        self.num_users = 100
        self.dataset = 'cifar100'
        self.epochs = 100
        self.frac = 0.1
        self.num_classes = 10 if self.dataset == 'cifar10'or self.dataset =='cifar10iid' else 100
        # self.num_classes = 10
        self.local_bs = 50
        self.local_ep = 5
        self.lr = 0.01  # learning rate
        self.optimizer = 'sgd'
        self.verbose = 1
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = 'ResNet18'
        self.n_shards = 100
        self.n_imgs = 500


args = Arg()

# __main__
start_time = time.time()
# using CPU
torch.cuda.set_device(0)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
device = args.device
print('deviece:', device)
print('Dataset:', args.dataset)
print('Model:', args.model)
print('The Sampling is n_shards=', args.n_shards,'n_imgs=',args.n_imgs)
# global_model = CNNCifar(args)
# if args.trainmodel == 'cifar10':
#     global_model = CNNCifar10(args)

global_model = ResNet18(ResBlock, args)
global_model.to(device)
global_model.train()
print(global_model)
global_weights = global_model.state_dict()
train_dataset, test_dataset, user_groups = get_dataset(args)

# Training
train_loss, train_accuracy, test_accuracy = [], [],[]
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0

for epoch in tqdm(range(args.epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch + 1} |\n')

    global_model.train()
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    for idx in idxs_users:
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx])
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # update global weights
    global_weights = average_weights(local_weights)

    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx])
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc) / len(list_acc))
    if hasattr(torch.cuda, 'empty_cache'):
        torch.cuda.empty_cache()

    # print global training loss after every 'i' rounds
    # if (epoch + 1) % print_every == 0:
    if 1:
        print(f' \nAvg Training Stats after {epoch + 1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100 * train_accuracy[-1]))
    test_acc, test_loss = test_inference(args, global_model, test_dataset)
    test_accuracy.append(test_acc)
    print("Test Accuracy: {:.2f}%".format(100 * test_acc))
# Test inference after completion of training
test_acc, test_loss = test_inference(args, global_model, test_dataset)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100 * train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100 * test_acc))
print('\n Total Run Time: {0:0.4f}'.format(time.time() - start_time))


# PLOTTING (optional)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

 # Plot Loss curve
plt.figure()
plt.title('Training Loss vs Communication rounds')
plt.plot(range(len(train_loss)), train_loss, color='r')
plt.ylabel('Training loss')
plt.xlabel('Communication Rounds')
plt.savefig('fed_{}_{}_{}_C[{}]_E[{}]_B[{}]_loss.png'.
            format(args.dataset, args.model, args.epochs, args.frac,
                     args.local_ep, args.local_bs))

# Plot Average Accuracy vs Communication rounds
plt.figure()
plt.title('Average Accuracy vs Communication rounds')
plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
plt.ylabel('Average Accuracy')
plt.xlabel('Communication Rounds')
plt.savefig('fed_{}_{}_{}_C[{}]_E[{}]_B[{}]_NSHARDS[{}]_avg_acc.png'.
            format(args.dataset, args.model, args.epochs, args.frac,
                     args.local_ep, args.local_bs, args.n_shards))
plt.figure()
plt.title('Test Accuracy vs Communication rounds')
plt.plot(range(len(test_accuracy)), test_accuracy, color='k')
plt.ylabel('Test Accuracy')
plt.xlabel('Communication Rounds')
plt.savefig('fed_{}_{}_{}_C[{}]_E[{}]_B[{}]_NSHARDS[{}]_test_acc.png'.
            format(args.dataset, args.model, args.epochs, args.frac,
                     args.local_ep, args.local_bs, args.n_shards))

deviece: cuda
Dataset: cifar100
Model: ResNet18
The Sampling is n_shards= 100 n_imgs= 500
ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 


Extracting ../data/cifar100/cifar-100-python.tar.gz to ../data/cifar100/
Files already downloaded and verified


  0%|          | 0/100 [00:00<?, ?it/s]


 | Global Training Round : 1 |



/content/drive/My Drive/Colab Notebooks/FedAvg/MyFedavg/sampling.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)


...

 99%|█████████▉| 99/100 [2:06:57<01:16, 76.71s/it]

Test Accuracy: 6.96%

 | Global Training Round : 100 |

| Global Round : 99 | Local Epoch : 0 | [0/800 (0%)]	Loss: 1.896882
| Global Round : 99 | Local Epoch : 0 | [500/800 (62%)]	Loss: 0.129703
| Global Round : 99 | Local Epoch : 1 | [0/800 (0%)]	Loss: 0.005296
| Global Round : 99 | Local Epoch : 1 | [500/800 (62%)]	Loss: 0.035030
| Global Round : 99 | Local Epoch : 2 | [0/800 (0%)]	Loss: 0.004642
| Global Round : 99 | Local Epoch : 2 | [500/800 (62%)]	Loss: 0.003213
| Global Round : 99 | Local Epoch : 3 | [0/800 (0%)]	Loss: 0.003180
| Global Round : 99 | Local Epoch : 3 | [500/800 (62%)]	Loss: 0.001991
| Global Round : 99 | Local Epoch : 4 | [0/800 (0%)]	Loss: 0.000838
| Global Round : 99 | Local Epoch : 4 | [500/800 (62%)]	Loss: 0.001730
| Global Round : 99 | Local Epoch : 0 | [0/800 (0%)]	Loss: 3.439413
| Global Round : 99 | Local Epoch : 0 | [500/800 (62%)]	Loss: 0.067841
| Global Round : 99 | Local Epoch : 1 | [0/800 (0%)]	Loss: 0.092393
| Global Round : 99 | Local Epoch : 1 | [5

100%|██████████| 100/100 [2:08:14<00:00, 76.94s/it]

Test Accuracy: 5.11%


 
 Results after 100 global rounds of training:
|---- Avg Train Accuracy: 10.00%
|---- Test Accuracy: 5.11%

 Total Run Time: 7718.4269
